In [ ]:
!pip install requests beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import json
import csv
from bs4 import BeautifulSoup
from datetime import datetime
import string
import re

# turnbackhoax.id

In [ ]:
def scrape_turnbackhoax():
    url = 'https://turnbackhoax.id/wp-json/wp/v2/posts'
    response = requests.get(url)
    data = response.json()
    
    dataset = []
    for item in data:
        label = 'SALAH'
        headline = item['title']['rendered'].replace('[SALAH]', '').strip()
        body_html = item['content']['rendered']
        
        soup = BeautifulSoup(body_html, 'html.parser')
        body = soup.get_text()
        
        dataset.append({'label': label, 'Headline': headline, 'Body': body})
    
    return dataset

def clean_text(text):
    # Remove Unicode decimal codes
    text = re.sub(r'&#\d+;', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove special characters and symbols
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    return text

def save_to_csv(dataset):
    now = datetime.now()
    timestamp = now.strftime('%Y-%m-%d_%H-%M-%S')
    filename = f'turnbackhoax_dataset_{timestamp}.csv'
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['label', 'Headline', 'Body'])
        writer.writeheader()
        
        cleaned_dataset = []
        for data in dataset:
            cleaned_data = {
                'label': data['label'],
                'Headline': clean_text(data['Headline']),
                'Body': clean_text(data['Body'])
            }
            cleaned_dataset.append(cleaned_data)
        
        writer.writerows(cleaned_dataset)
    
    print('Data saved to', filename)

# Scrape data from turnbackhoax.id
all_data = []
for _ in range(10): #mau berapa data yg di scrap?
    data = scrape_turnbackhoax()
    all_data.extend(data)

# Save data to CSV file
save_to_csv(all_data)

Data saved to turnbackhoax_dataset_2023-05-18_06-32-08.csv
